In [2]:
import urllib.request  as urllib2
from bs4 import BeautifulSoup
import re
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from joblib import Parallel, delayed
import itertools
import json
from urllib.request import urlopen
import requests
import numpy as np
import multiprocessing
import pickle

In [40]:
data = people_links
with open('people_links_data.pickle', 'wb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(data, f, pickle.HIGHEST_PROTOCOL)

In [3]:
with open('people_links_data.pickle', 'rb') as f:
    # The protocol version used is detected automatically, so we do not
    # have to specify it.
    people_links_loaded_data = pickle.load(f)

In [45]:
data = repos_data_no_commits
with open('repos_no_commits_data.pickle', 'wb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(data, f, pickle.HIGHEST_PROTOCOL)

In [4]:
with open('repos_no_commits_data.pickle', 'rb') as f:
    # The protocol version used is detected automatically, so we do not
    # have to specify it.
    repos_no_commit_loaded_data = pickle.load(f)

In [5]:
len(repos_no_commit_loaded_data)

1855

In [6]:
# Se obtiene la página evitando el error de muchas consultas seguidas
def get_page(url):    
    could_read = False
    counter = 0
    while(could_read == False):
        try:
            page  = urllib2.urlopen(url)
            could_read = True
        except:
            print('Page not available '+url+' , will sleep a while')
            time.sleep(10)    
            counter += 1
            if (counter > 3):
                return []
    page_read = page.read()
    page_data = BeautifulSoup(page_read, 'lxml')
    page_text = page_data.decode('utf-8')
    urllib2.urlcleanup()
    return page_text

In [7]:
# Se obtienen los links de todas las páginas de usuarios
def get_people_pages(organization):
    url_path = ('https://github.com/orgs/'+organization+'/people')
    page_text = get_page(url_path)
    
    pages = re.findall('<a href="/orgs/'+organization+'/people\?page=(.*?)"',page_text.replace("\n",""))    
    if(len(pages) == 0):
        pages = 1
    else:
        pages = int(pages[len(pages)-1])
    links = []
    for page in range(pages):
        links.append('https://github.com/orgs/'+organization+'/people?page='+str((int(page)+1)))                
    return links

In [8]:
# Se obtienen los links de todas las páginas de los repositorios de un usuario
def get_repositories_pages(profile):    
    page_text = get_page(profile+'?tab=repositories')
    page_text = re.sub('\s+',' ',page_text)
    pages = re.findall('<div class="pagination">(.*?)</div>',page_text.replace("\n",""))
    links = []
    if(len(pages)!=0):        
        page_numbers = re.findall('tab=repositories"> (\d+) </a>',pages[0])
        page_numbers.append(re.findall('tab=repositories" rel="next"> (\d+) </a>',pages[0])[0])        
        page_numbers = [int(i) for i in page_numbers]
        page_number = max(page_numbers)                        
        for page in range(page_number):
            links.append(profile+'?page='+str((int(page)+1))+'&tab=repositories')        
    else:
        links.append(profile+'?tab=repositories')
    return links

In [9]:
# Se obtienen los nombres de los usuarios
def get_people_users(organization):
    links = get_people_pages(organization)
    people_users = []
    for link in links:
        url_path = link                            
        page_text = get_page(url_path)

        users = re.findall('<a class="css-truncate-target f4" href="/(.*?)">',page_text.replace("\n",""))        
        for user in users:
            people_users.append(user)
    return people_users

In [10]:
def get_people_links_sub(link):
    s_people_links = []
    url_path = link                            
    page_text = get_page(url_path)

    users = re.findall('<a class="css-truncate-target f4" href="(.*?)">',page_text.replace("\n",""))        
    for user in users:
        s_people_links.append('https://github.com'+user)
        
    return s_people_links

In [11]:
def get_repo_commits_data(repository_data):
    repo_data = []
    user_name, repo_name, repo_language, repo_type = repository_data
    try:
        rep_info = get_repository_commits('https://github.com/'+user_name+'/'+repo_name,repo_type)
    except:
        return []
    
    if (repo_type == 'forked'):
        total_commits = rep_info[0]
        orig_owner = rep_info[1]
    else:
        total_commits = rep_info
    
    
    repo_data.append(user_name)
    repo_data.append(repo_name)
    repo_data.append(repo_language)
    repo_data.append(total_commits)
    # personal contributions
    extra_time = 0
    could_read = False
    
    while(could_read == False):
        try:
            if (repo_type == 'forked'):
                response = requests.get('https://api.github.com/repos/'+orig_owner+'/'+repo_name+'/stats/contributors',
                                    auth=('akawa94', '2f4b192e2f583e3379c54dad5c31bfc9217e7e85'))
            else:
                response = requests.get('https://api.github.com/repos/'+user_name+'/'+repo_name+'/stats/contributors',
                                    auth=('akawa94', '2f4b192e2f583e3379c54dad5c31bfc9217e7e85'))
            data = response.json()
            could_read = True
        
        except:
            response = requests.get('https://api.github.com/rate_limit',
                                    auth=('akawa94', '2f4b192e2f583e3379c54dad5c31bfc9217e7e85'))
            if int(response.json()['resources']['core']['remaining']) > 0:
                repo_data.append(0)
                repo_data.append(0)
                repo_data.append(0)
                repo_data.append(repo_type)
                return repo_data
            print("Caught in while loop, waiting for github api to refresh -- "+user_name+'_'+repo_name)
            time.sleep(15+extra_time)
            extra_time +=5

    try:
        json_data = [x for x in data if x['author']['login']==user_name]
    except:
        repo_data.append(0)
        repo_data.append(0)
        repo_data.append(0)
        repo_data.append(repo_type)
        return repo_data
    if (len(json_data) == 0):
        repo_data.append(0)
        repo_data.append(0)
        repo_data.append(0)
        repo_data.append(repo_type)
        return repo_data
    json_data = json_data[0]
    personal_commits = sum(list(map(lambda x:x['c'],json_data['weeks'])))
    lines_added = sum(list(map(lambda x:x['a'],json_data['weeks'])))
    lines_removed = sum(list(map(lambda x:x['d'],json_data['weeks'])))
     
    
    repo_data.append(personal_commits)
    repo_data.append(lines_added)
    repo_data.append(lines_removed)
    repo_data.append(repo_type)
    print('-----> Finished processing '+user_name+'_'+repo_name)
    return repo_data
    
    

In [12]:
def get_repo_data_no_commits(repository, repo_type):
    repo_data = []                      
    repo_link = get_repositories_links([repository])[0]
    repo_name = get_repositories_names([repository])[0]
    user_name = re.findall('com/(.*?)/'+repo_name+'$',repo_link)[0]    
    print("Processing "+user_name+" "+repo_name)
    repo_language = get_language_per_repository_block([repository])        
    print("-----> Finished processing "+user_name+" "+repo_name)
    repo_data.append(user_name)
    repo_data.append(repo_name)
    repo_data.append(repo_language)
    repo_data.append(repo_type)
    
    return repo_data

In [13]:
def get_user_repositories_data_no_commits(profile):
    if (profile=='https://github.com/m3drano'):
        return []
    print(profile+' from beginning')
    links = get_repositories_pages(profile)
    forked_repos_data = []
    own_repos_data = []
    for link in links:
        page_text = get_page(link)
        forked_repositories = re.findall('<li class="col-12 d-block width-full py-4 border-bottom public fork" itemprop="owns" itemscope="" itemtype="http://schema.org/Code">(.*?)</li>',page_text.replace("\n",""))
        own_repositories = re.findall('<li class="col-12 d-block width-full py-4 border-bottom public source" itemprop="owns" itemscope="" itemtype="http://schema.org/Code">(.*?)</li>',page_text.replace("\n",""))
        for forked_repo in forked_repositories:
            forked_repos_data.append(get_repo_data_no_commits(forked_repo,'forked'))
        for own_repo in own_repositories:
            own_repos_data.append(get_repo_data_no_commits(own_repo,'own'))
    print(profile+' from ending')
    return forked_repos_data+own_repos_data

In [14]:
# Se obtienen los "repositories blocks" de los repositorios forkeados  
def get_forked_repositories_blocks(profile):
    print(profile+' from beginning')
    links = get_repositories_pages(profile)
    repos_links = []
    for link in links:
        url_path = link
        page_text = get_page(url_path)
    
        forked_repositories = re.findall('<li class="col-12 d-block width-full py-4 border-bottom public fork" itemprop="owns" itemscope="" itemtype="http://schema.org/Code">(.*?)</li>',page_text.replace("\n",""))
        for forked_repo in forked_repositories:
            repos_links.append(forked_repo)
    print(profile+' from ending')
    return repos_links

# Se obtienen los "repositories blocks" de los repositorios propios 
def get_own_repositories_blocks(profile):    
    links = get_repositories_pages(profile)
    repos_links = []
    for link in links:
        url_path = link
        page_text = get_page(url_path)
        
        own_repositories = re.findall('<li class="col-12 d-block width-full py-4 border-bottom public source" itemprop="owns" itemscope="" itemtype="http://schema.org/Code">(.*?)</li>',page_text.replace("\n",""))
        for own_repo in own_repositories:
            repos_links.append(own_repo)
    return repos_links

In [15]:
# Obtiene los links de los repositorios apartir de los "repositories blocks"
def get_repositories_links(repositories):
    links = []
    for repository in repositories:        
        repository_name = re.findall('<a href="(.*?)" itemprop="name codeRepository">',repository)        
        links.append('https://github.com'+repository_name[0])
    return links

def get_repositories_names(repositories):
    names = []    
    for repository in repositories:        
        if(len(repository) != 0):            
            name = re.findall('itemprop="name codeRepository">(.*?)</a>',repository)               
            names.append(name[0].strip())
    return names

In [68]:
[].replace("\n",'')

AttributeError: 'list' object has no attribute 'replace'

In [16]:
def get_language_per_repository_block(repository_block):
    language = re.findall('<span class="mr-3" itemprop="programmingLanguage">(.*?)</span>',repository_block[0])
    if(len(language)==0):
        language.append('Unknown')
    return language[0].strip()

In [17]:
def get_repository_commits(repository_link, repo_type):
    page_text = get_page(repository_link)
    sup_page_text = re.sub('\s+',' ',page_text.replace("\n","")) 
    page_text = re.findall('<li class="commits">(.*?)</li>',sup_page_text)
    if(len(page_text)>0):
        commits = re.findall('<span class="num text-emphasized"> (\d+(\,\d+)?) </span>',page_text[0])
        if(len(commits)>0):
            commits = commits[0][0]
    else:
        commits = 0
    if (repo_type == 'forked'):
        orig_owner = re.findall('<span class="fork-flag">(.*?)</span>',sup_page_text)[0]
        orig_owner = re.findall('<a href="(.*?)">',orig_owner)[0].split('/')[1]
        return [commits,orig_owner]
    return commits

In [18]:
def get_profile_data(profile):
    print(profile+' before reading')
    page_text = get_page(profile)
    print(profile+' after reading')
    page_text = re.sub('\s+',' ',page_text.replace("\n",""))
    profile_data = []

    nav = re.findall('<nav(.*?)</nav>',page_text)
    location = re.findall('<ul class="vcard-details border-top border-gray-light py-3">(.*?)</ul>',page_text)
    location = re.findall('<span class="p-label">(.*?)</span>',location[0])
    if(len(location)>0):
        location = location[0]
    else:
        location = 'Unknown'

    tags = re.findall('<span class="Counter"> (\d+(\.\d+)?k?) </span>',nav[1])    
    num_repos = tags[0][0]
    num_stars = tags[1][0]
    num_followers = tags[2][0]
    num_following = tags[3][0]

    user_name = re.findall('com/(.*?)$',profile)[0]

    profile_data.append(user_name)
    profile_data.append(location)
    profile_data.append(num_repos)
    profile_data.append(num_stars)
    profile_data.append(num_followers)
    profile_data.append(num_following)
    
    return profile_data

In [19]:
def get_user_repository_data_v2(param):
    repository = param[0]
    repo_type = param[1]
    
    repo_lang = []                        
    repo_link = get_repositories_links([repository])[0]
    repo_name = get_repositories_names([repository])[0]
    user_name = re.findall('com/(.*?)/'+repo_name+'$',repo_link)[0]    
    repo_language = get_language_per_repository_block([repository])        
    
    
    # total commits
    if (repo_type == 'forked'):
        rep_info = get_repository_commits('https://github.com/'+user_name+'/'+repo_name,repo_type)
        total_commits = rep_info[0]
        orig_owner = rep_info[1]
    else:
        total_commits = get_repository_commits('https://github.com/'+user_name+'/'+repo_name,repo_type)
    
    repo_lang.append(user_name)
    repo_lang.append(repo_name)
    repo_lang.append(repo_language)
    repo_lang.append(total_commits)
    
    
    
    # personal contributions
    extra_time = 0
    could_read = False
    while(could_read == False):
        try:
            if (repo_type == 'forked'):
                response = requests.get('https://api.github.com/repos/'+orig_owner+'/'+repo_name+'/stats/contributors',
                                    auth=('akawa94', '2f4b192e2f583e3379c54dad5c31bfc9217e7e85'))
            else:
                response = requests.get('https://api.github.com/repos/'+user_name+'/'+repo_name+'/stats/contributors',
                                    auth=('akawa94', '2f4b192e2f583e3379c54dad5c31bfc9217e7e85'))
            data = response.json()
            could_read = True
        
        except:
            response = requests.get('https://api.github.com/rate_limit',
                                    auth=('akawa94', '2f4b192e2f583e3379c54dad5c31bfc9217e7e85'))
            if int(response.json()['resources']['core']['remaining']) > 0:
                repo_lang.append(0)
                repo_lang.append(0)
                repo_lang.append(0)
                repo_lang.append(repo_type)
                return repo_lang
            print("Caught in while loop, waiting for github api to refresh -- "+user_name+'_'+repo_name)
            time.sleep(15+extra_time)
            extra_time +=5

    try:
        json_data = [x for x in data if x['author']['login']==user_name]
    except:
        repo_lang.append(0)
        repo_lang.append(0)
        repo_lang.append(0)
        repo_lang.append(repo_type)
        return repo_lang
    if (len(json_data) == 0):
        repo_lang.append(0)
        repo_lang.append(0)
        repo_lang.append(0)
        repo_lang.append(repo_type)
        return repo_lang
    json_data = json_data[0]
    personal_commits = sum(list(map(lambda x:x['c'],json_data['weeks'])))
    lines_added = sum(list(map(lambda x:x['a'],json_data['weeks'])))
    lines_removed = sum(list(map(lambda x:x['d'],json_data['weeks'])))
     
    
    repo_lang.append(personal_commits)
    repo_lang.append(lines_added)
    repo_lang.append(lines_removed)
    repo_lang.append(repo_type)
    print('Finished processing '+user_name+'_'+repo_name)
    return repo_lang

In [20]:
def get_user_repository_data(repository,repo_type):
    options = Options()
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument('--no-sandbox')
    
    repo_lang = []                        
    repo_link = get_repositories_links([repository])[0]
    repo_name = get_repositories_names([repository])[0]
    user_name = re.findall('com/(.*?)/'+repo_name+'$',repo_link)[0]    
    repo_language = get_language_per_repository_block([repository])        
    driver = webdriver.Chrome('/home/a20114261/chromedriver',chrome_options=options)
    driver.get('https://github.com/'+user_name+'/'+repo_name+'/graphs/contributors')
    src = driver.page_source
    page_data = BeautifulSoup(src, 'lxml')
    page_text = page_data.decode('utf-8')

    data = re.sub('\s+',' ',page_text.replace("\n",""))
    #print(data)
    data = re.findall('<h3 class="border-bottom p-2 lh-condensed">(.*?)</h3>',page_text.replace("\n",""))        
    #print(data)
    data = ' '.join(data)        
    #print(data)
    data = re.sub('\s+',' ',data)        
    #print(data)
    data = re.findall('<a class="link-gray text-normal" href="https://github.com/'+user_name+'/'+repo_name+'/commits\?author='+user_name+'">(.*?)</span> </span>',data)                
    #print(data)
    
    if len(data)>0:            
        commits = re.findall('(\d+(\,\d+)?) commit',data[0])[0]
        if len(commits)>0:
            commits = commits[0].split()[-1]
        else:
            commits = -1

        lines_added = re.findall('>(.+) \+\+',data[0])
        if len(lines_added)>0:
            lines_added = lines_added[0].split()[-1]
        else:
            lines_added = -1

        lines_removed = re.findall('>(.+) \-\- $',data[0])
        if len(lines_removed)>0:
            lines_removed = lines_removed[0].split()[-1]
        else:
            lines_removed = -1
    else:            
        commits = 0
        lines_added = 0
        lines_removed = 0 
    repo_lang.append(user_name)
    repo_lang.append(repo_name)
    repo_lang.append(repo_language)
    repo_lang.append(commits)
    repo_lang.append(lines_added)
    repo_lang.append(lines_removed)
    repo_lang.append(repo_type)
    print(user_name+'_'+repo_name)
    return repo_lang

In [21]:
def get_final_data(all_repositories):
    repos_langs = []
    for repositories in all_repositories:
        for forked_repository in repositories[0]:
            if forked_repository != []:                  
                repo_lang = get_user_repository_data(forked_repository,'forked')                
                repos_langs.append(repo_lang)
        for own_repository in repositories[1]:
            if own_repository != []:
                repo_lang = get_user_repository_data(own_repository,'own')                
                repos_langs.append(repo_lang)
    return repos_langs

In [21]:
url = 'https://google.com'
page  = urllib2.urlopen(url)

In [ ]:
organization = 'google'
links = get_people_pages(organization)
print(len(links))

In [ ]:
# Se obtienen los perfiles de los usuarios
#def get_people_links_v2(organization):
# organization = 'google'
# links = get_people_pages(organization)
# print(len(links))
people_links = []
for i in range(0,len(links),40):
    if (i+40)<len(links):
        top_lim = i+40
    else:
        top_lim = len(links)
    pool = multiprocessing.Pool(20)
    people_links += pool.map(get_people_links_sub,links[i:top_lim])
    #time.sleep(5)
    print(i)
    print(top_lim)
    print('Finished')
#people_links = list(itertools.chain.from_iterable(people_links))
    #return people_links

In [ ]:
people_links = list(itertools.chain.from_iterable(people_links))

In [ ]:
for i in range(0,len(people_links),80):
    print(i)

In [ ]:
get_profile_data(people_links[80])

In [ ]:
#profiles_data = []
for i in range(1700,len(people_links),20):
    if i+20>len(people_links):
        top_lim = len(people_links)
    else:
        top_lim = i+20
    profiles_data += Parallel(n_jobs=20)(map(delayed(get_profile_data),people_links[i:top_lim]))
    time.sleep(5)
    print("Analyzed from index "+str(i)+" to "+str(top_lim))

In [ ]:
import copy
backup_profiles_data = copy.deepcopy(profiles_data)
len(profiles_data)

In [ ]:
pd.set_option('display.max_rows', 500)
df2 = pd.DataFrame(data=profiles_data,index=None,columns=["User","Location","Num_Repos","Num_Stars","Num_Followers","Num_Following"])
print(df2.shape)
df2.head()

In [ ]:
df2.to_csv('google_userbase.csv',index=False)

In [31]:
df2 = pd.read_csv('google_userbase.csv')

In [34]:
df2.head()

,User,Location,Num_Repos,Num_Stars,Num_Followers,Num_Following
0,3rf,New York City,27,40,346,0
1,aaroey,Unknown,2,89,203,26
2,aarongable,San Francisco,17,0,661,0
3,aaronj1335,"Austin, Texas",104,104,407,23
4,aarontp,Unknown,10,98,200,0


In [ ]:
%%time
test_result = get_user_repositories_data_no_commits('https://github.com/danijar')

In [ ]:
%%time
test_result = []
for i in range(0,step):
    test_result += get_user_repositories_data_no_commits(people_links[i])
    print("Finished for index "+str(i))

In [22]:
people_links = people_links_loaded_data
repos_data_no_commits = repos_no_commit_loaded_data

3390

In [23]:
def get_remaining_api_requests():
    response = requests.get('https://api.github.com/rate_limit',
                                    auth=('akawa94', '2f4b192e2f583e3379c54dad5c31bfc9217e7e85'))
    return response.json()['rate']['remaining']

3390

In [39]:
step = 2
#repos_data_no_commits = []
for j in range(1312,len(people_links),step):   
    pool = multiprocessing.Pool(step)
    if (j+step>len(people_links)):
        top_lim = len(people_links)
    else:
        top_lim = j+step
    repos_data_no_commits += pool.map(get_user_repositories_data_no_commits,people_links[j:top_lim])
    print("before preventing memory leak")
    pool.close()
    pool.join()
    print("======================================================================================")
    print("======================================================================================")
    print("======================================================================================")
    print(j)
    print(top_lim)
    print("Now sleeping for 20 seconds...")
    time.sleep(20)
    print("Right back on track!")
    print("======================================================================================")
    print("======================================================================================")
    print("======================================================================================")

https://github.com/paullewis from beginning
https://github.com/paulirish from beginning
Processing paullewis project-health
-----> Finished processing paullewis project-health
Processing paullewis tensorboard
-----> Finished processing paullewis tensorboard
Processing paullewis shaka-player
-----> Finished processing paullewis shaka-player
Processing paullewis WebFundamentals
-----> Finished processing paullewis WebFundamentals
Processing paullewis samples
-----> Finished processing paullewis samples
Processing paullewis preact
-----> Finished processing paullewis preact
Processing paullewis vrview
-----> Finished processing paullewis vrview
Processing paullewis lighthouse
-----> Finished processing paullewis lighthouse
Processing paullewis Fireworks
-----> Finished processing paullewis Fireworks
Processing paullewis BuildTest
-----> Finished processing paullewis BuildTest
Processing paullewis Boundarizr
-----> Finished processing paullewis Boundarizr
Processing paullewis music-dna
---

In [49]:
repos_data_no_commits[0]

[['3rf', 'material2', 'TypeScript', 'forked'],
 ['3rf', 'xterm.js', 'TypeScript', 'forked'],
 ['3rf', 'DefinitelyTyped', 'TypeScript', 'forked'],
 ['3rf', 'material', 'JavaScript', 'forked'],
 ['3rf', 'sample', 'Python', 'forked'],
 ['3rf', 'mongo-c-driver', 'C', 'forked'],
 ['3rf', 'perf', 'JavaScript', 'forked'],
 ['3rf', 'keyval', 'Go', 'forked'],
 ['3rf', '2015-talks', 'Unknown', 'forked'],
 ['3rf', 'mongo', 'C++', 'forked'],
 ['3rf', 'openssl', 'Go', 'forked'],
 ['3rf', 'mgo', 'Go', 'forked'],
 ['3rf', 'goxc', 'Go', 'forked'],
 ['3rf', 'mongo-lint', 'Go', 'forked'],
 ['3rf', 'mongo-perf', 'JavaScript', 'forked'],
 ['3rf', 'mongo-python-driver', 'Python', 'forked'],
 ['3rf', 'tater', 'PHP', 'forked'],
 ['3rf', 'codecoroner', 'Go', 'own'],
 ['3rf', 'evg-iframe', 'Go', 'own'],
 ['3rf', 'insertbench', 'Python', 'own'],
 ['3rf', 'evergreen-test-project', 'Shell', 'own'],
 ['3rf', 'winpass', 'Go', 'own'],
 ['3rf', 'solarpass', 'Go', 'own'],
 ['3rf', 'gophermark', 'Go', 'own'],
 ['3rf', 

In [24]:
# house cleaning
for i in range(len(repos_data_no_commits)-1,-1,-1):
    if (len(repos_data_no_commits[i])==0):
        repos_data_no_commits.pop(i)
        print('xd')
    for j in range(len(repos_data_no_commits[i])-1,-1,-1):
        if (len(repos_data_no_commits[i][j])==0):
            repos_data_no_commits[i].pop(j)
            print('lol')
        

xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd
xd


In [36]:
pre_repos_data = list(itertools.chain.from_iterable(repos_data_no_commits))

In [37]:
len(pre_repos_data)

48243

In [59]:
for i in range(0,20,10):
    print(i)

0
10


In [71]:
'repos_data_'+str(len(repos_data))+'_of_'+str(len(pre_repos_data))+'.pickle'

'repos_data_0_of_48243.pickle'

In [24]:
get_remaining_api_requests()

5000

In [ ]:
repos_data = []

In [59]:
len(repos_data)

1855

In [ ]:
%%time
step = 4500
for j in range(6355,len(pre_repos_data),step):
    tries_flag = True
    while (tries_flag == True):
        if (get_remaining_api_requests() >= step):
            break
        else:
            print("Now sleeping for till remaining tries are 5000 again ...")
            time.sleep(300) #sleeping for 5 minutes
    print("Right back on track!")
    print("======================================================================================")
    print("======================================================================================")
    print("======================================================================================")
    pool = multiprocessing.Pool(10)
    if (j+step>len(pre_repos_data)):
        top_lim = len(pre_repos_data)
    else:
        top_lim = j+step
    repos_data += pool.map(get_repo_commits_data,pre_repos_data[j:top_lim])
    print("before preventing memory leak")
    pool.close()
    pool.join()
    with open('repos_data_'+str(len(repos_data))+'_of_'+str(len(pre_repos_data))+'.pickle', 'wb') as f:
        # Pickle the 'data' dictionary using the highest protocol available.
        pickle.dump(repos_data, f, pickle.HIGHEST_PROTOCOL)
    print("======================================================================================")
    print("======================================================================================")
    print("======================================================================================")
    print(j)
    print(top_lim)

Right back on track!
-----> Finished processing clelland_cordova-cli
-----> Finished processing cgrushko_bazel
-----> Finished processing chenshay_amphtml
-----> Finished processing chr1sj0nes_TensorComprehensions
-----> Finished processing cchamberlain_cryptkeeper
-----> Finished processing cchamberlain_cpath
-----> Finished processing cchamberlain_tig-link
-----> Finished processing clelland_django
-----> Finished processing chinmaygarde_flutter
-----> Finished processing chingor13_opencensus-php
-----> Finished processing cgrushko_protobuf
-----> Finished processing chriscox_material-components-ios
-----> Finished processing cgrushko_iTerm2
-----> Finished processing chriscox_nimbus
-----> Finished processing clement_request
-----> Finished processing chingor13_hex
-----> Finished processing ChrisMondok_mta-gtfs
-----> Finished processing cdata_umd
-----> Finished processing clement_connect-redis
-----> Finished processing chingor13_community
-----> Finished processing chaopeng_libc

In [25]:
with open('repos_data_48243_of_48243.pickle','rb') as f:
    repos_data = pickle.load(f)

48243


In [26]:
pd.set_option('display.max_rows', 500)
df = pd.DataFrame(data=repos_data,index=None,columns=["User","Repository","Language","TotalCommits","PersonalCommits","Lines_Added","Lines_Removed","Type"])
print(df.shape)
df.to_csv('google_base.csv', index=False)

(48243, 8)


In [27]:
df.shape

(48243, 8)

In [28]:
language_columns = list(set(list(df['Language'])))
unique_users = list(set(list(df["User"])))

In [47]:
for i in range(len(language_columns)-1,-1,-1):
    if (language_columns[i] == None ):
        language_columns.pop(i)
        print('xd')
        
for i in range(len(unique_users)-1,-1,-1):
    if (unique_users[i] == None ):
        unique_users.pop(i)
        print('xd')

In [37]:
def prefered_language(username,language_columns):
    count = []
    for language in language_columns:
        query_str = 'User=="'+username+'" and Language=="'+str(language)+'"'
        print(query_str)
        count.append(len(list(df.query(query_str)['Language'])))
    return language_columns[count.index(max(count))]

In [48]:
df_columns = list(df2.keys())+["Organization","PreferedLanguage"]\
    +list(map(lambda x:x+'_Forked_TotalCommits',language_columns))\
    +list(map(lambda x:x+'_Own_TotalCommits',language_columns))\
    +list(map(lambda x:x+'_Forked_PersonalCommits',language_columns))\
    +list(map(lambda x:x+'_Own_PersonalCommits',language_columns))\
    +list(map(lambda x:x+'_Forked_TotalProjects',language_columns))\
    +list(map(lambda x:x+'_Own_TotalProjects',language_columns))\
    +list(map(lambda x:x+'_Forked_TotalAdditions',language_columns))\
    +list(map(lambda x:x+'_Own_TotalAdditions',language_columns))\
    +list(map(lambda x:x+'_Forked_TotalDeletes',language_columns))\
    +list(map(lambda x:x+'_Own_TotalDeletes',language_columns))

In [49]:
df_columns

['User',
 'Location',
 'Num_Repos',
 'Num_Stars',
 'Num_Followers',
 'Num_Following',
 'Organization',
 'PreferedLanguage',
 'Matlab_Forked_TotalCommits',
 'JavaScript_Forked_TotalCommits',
 'C#_Forked_TotalCommits',
 'Cuda_Forked_TotalCommits',
 'Kotlin_Forked_TotalCommits',
 'CoffeeScript_Forked_TotalCommits',
 'Propeller Spin_Forked_TotalCommits',
 'XML_Forked_TotalCommits',
 'PostScript_Forked_TotalCommits',
 'Python_Forked_TotalCommits',
 'Erlang_Forked_TotalCommits',
 'Bluespec_Forked_TotalCommits',
 'Yacc_Forked_TotalCommits',
 'Processing_Forked_TotalCommits',
 'Eagle_Forked_TotalCommits',
 'Inform 7_Forked_TotalCommits',
 'KiCad Layout_Forked_TotalCommits',
 'HCL_Forked_TotalCommits',
 'Julia_Forked_TotalCommits',
 'Unknown_Forked_TotalCommits',
 'GDB_Forked_TotalCommits',
 'Handlebars_Forked_TotalCommits',
 'LOLCODE_Forked_TotalCommits',
 'Alloy_Forked_TotalCommits',
 'Nix_Forked_TotalCommits',
 'M4_Forked_TotalCommits',
 'Objective-C++_Forked_TotalCommits',
 'Ragel in Ruby H

In [52]:
def transform_data(user, language_columns, repo_type, q_type, process_type):
    results = [0  for x in language_columns]
    super_query = df.query('User == "'+user+'" and Type == "'+repo_type+'"')
    for i in range(0,len(language_columns)):
        language = language_columns[i]
        query_str = 'Language=="'+str(language)+'"'
        query_result = list(super_query.query(query_str)[q_type])
        if (process_type == "sum"):
            for j in range(0,len(query_result)):
                try:
                    query_result[j] = int(query_result[j].replace(',',''))
                except:
                    query_result[j] = int(query_result[j])
            results[i] = sum(query_result)
        elif (process_type == "count"):
            results[i] = len(query_result)
    
    return results
    

In [53]:
def transform_mask(user_dict, user, language_columns, param_list):
    repo_type = param_list[0]
    q_type = param_list[1]
    process_type = param_list[2]
    additional_key = param_list[3]
    s_data = transform_data(user, language_columns,repo_type,q_type,process_type)
    s_keys = list(map(lambda x:x+additional_key,language_columns))
    for i in range(0,len(language_columns)):
        user_dict[s_keys[i]] = s_data[i]
    
    return user_dict

In [ ]:
%%capture
df_transformed = pd.DataFrame(index=None,columns=df_columns)
for user in unique_users:
    user_dict = df2.query('User == "'+user+'"').to_dict()
    print(user_dict)
    for key in user_dict:
        for s_key in user_dict[key]:
            user_dict[key] = user_dict[key][s_key]
    user_dict['PreferedLanguage'] = prefered_language(user,language_columns)
    user_dict['Organization'] = 'Google'
    
    
    param_list = [
        ['forked','TotalCommits',"sum",'_Forked_TotalCommits'], # origin forked total commits
        ['own','TotalCommits',"sum",'_Own_TotalCommits'], # origin own total commits
        ['forked','PersonalCommits',"sum",'_Forked_PersonalCommits'], # personal forked personal commits
        ['own','PersonalCommits',"sum",'_Own_PersonalCommits'], # personal own personal commits
        ['forked','PersonalCommits',"count",'_Forked_TotalProjects'], # forked total repositories
        ['own','PersonalCommits',"count",'_Own_TotalProjects'], # own total repositories
        ['forked','Lines_Added',"sum",'_Forked_TotalAdditions'],
        ['own','Lines_Added',"sum",'_Own_TotalAdditions'],
        ['forked','Lines_Removed',"sum",'_Forked_TotalDeletes'],
        ['own','Lines_Removed',"sum",'_Own_TotalDeletes'],
    ]
    
    for param in param_list:
        user_dict = transform_mask(user_dict, user, language_columns, param)
    print(user_dict['User'])
    df_transformed = df_transformed.append(user_dict, ignore_index=True)
    

In [63]:
df_transformed.head()

,User,Location,Num_Repos,Num_Stars,Num_Followers,Num_Following,Organization,PreferedLanguage,Matlab_Forked_TotalCommits,JavaScript_Forked_TotalCommits,...,Vim script_Own_TotalDeletes,Pure Data_Own_TotalDeletes,Crystal_Own_TotalDeletes,Brightscript_Own_TotalDeletes,R_Own_TotalDeletes,Pascal_Own_TotalDeletes,Fortran_Own_TotalDeletes,HTML_Own_TotalDeletes,ANTLR_Own_TotalDeletes,Turing_Own_TotalDeletes
0,mkjelland,Seattle,11,10,29,0,Google,Go,0,0,...,0,0,0,0,0,0,0,0,0,0
1,vahidk,California,9,61,400,40,Google,C++,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ssundarraj,"Mountain View, CA",50,373,149,127,Google,Python,0,35,...,0,0,0,0,0,0,0,0,0,0
3,bprashanth,Unknown,22,0,66,0,Google,Go,0,0,...,0,0,0,0,0,0,0,4,0,0
4,vinc3m1,"Oakland, CA",28,311,416,60,Google,Java,0,3,...,0,0,0,0,0,0,0,0,0,0


In [62]:
df_transformed.shape

(1746, 1728)

In [64]:
df_transformed.to_csv('google_transformed.csv',index=False)